In [1]:
import cv2
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import colorsys
import numpy as np
import random

In [2]:
MODEL_FILE = "checkpoints/modelPSPNet_rgb.20.keras"
TEST_DIRECTORY = "_data/PhenoBench/val/images"
IMAGE_SIZE = (473, 473)
COLOR_MASK_ALPHA = .3

In [19]:
model = keras.models.load_model(MODEL_FILE, compile=False)

In [20]:
files = os.listdir(TEST_DIRECTORY)
random.shuffle(files)

In [24]:
x = keras.utils.image_dataset_from_directory(TEST_DIRECTORY,
                                                 batch_size=1,
                                                 image_size=IMAGE_SIZE,
                                                 crop_to_aspect_ratio=True,
                                                 labels=None).map(lambda x0: x0 / 255)

Found 772 files.


In [ ]:
for image in x:
    #fig, axes = plt.subplot(2, 2)
    np_image = image.numpy().squeeze()
    print(np_image.shape)

    # output should have the shape (473, 473, 25)
    model_output = model(image, training=False).numpy().squeeze()
    print(np.unique(model_output))
    
    overlayed = np_image.copy()
    overlay = np_image.copy()
    
    fig = plt.figure()
    mask_axes = fig.subplots(5, 5)
    plt.setp(mask_axes, xticks=[], yticks=[])
    
    for ax, i in zip(mask_axes.flatten(), range(model_output.shape[2])):
        class_mask = model_output[:, :, i]
        #color = [int(round(i * 255)) for i in colorsys.hsv_to_rgb(i / model_output.shape[2], 1, 1)]
        color = colorsys.hsv_to_rgb(i / model_output.shape[2], 1, 1)
        overlay[class_mask == 1] = color
        ax.imshow(class_mask, cmap='gray', vmin=0, vmax=1)
        
    overlayed = cv2.addWeighted(overlay, COLOR_MASK_ALPHA, overlayed, 1 - COLOR_MASK_ALPHA, 0)
    cv2.imwrite("out/vis.jpg", overlayed)
    fig.show()
    plt.imshow(np_image)
    plt.show()
    
    break